# Decision Trees


### By the end of this lecture you will:
1. Describe in your own words the use cases, strengths and weaknesses of Decision Tree and Random Forest classifiers.
2. Describe the basic algorithm of Decision Tree construction.
3. Know what Gini entropy is.

# Decision trees
 
[Decision trees](http://en.wikipedia.org/wiki/Decision_tree_learning) are one of the most popular and widely used algorithms in industry today. Most classifiers (SVM, kNN, Naive Bayes) are great at giving you a (somewhat) accurate result, but are often black boxes in that it can be hard to interpret their parameters and thus *understand why* a certain instance was assigned a specific label. Regressions provide a middle ground in that it is possible to assign relationships between features and targets but it is often to assign a causal relationship amongst the inputs (Lasso/Elastic net is a way of getting at this problem). Decision trees are unique among any standard model in that they are both flexible and accurate while also being easily interpreted.

![c4.5](images/golftree.gif)

__INPUTS:__ Nominal (discrete) or Continuous

__OUTPUTS:__ Nominal (discrete) or Continuous

__(basically anything in and anything out)__


### Decision Tree Tradeoffs
#### Why Decision Trees

* Easily interpretable
* Handles missing values and outliers
* [non-parametric](http://en.wikipedia.org/wiki/Non-parametric_statistics#Non-parametric_models)/[non-linear](http://www.yaksis.com/static/img/02/cows_and_wolves.png)/model complex phenomena
* Computationally _cheap_ to ___predict___
* Can handle irrelevant features
* Mixed data (nominal and continuous)

#### Why Not Decision Trees

* Computationally _expensive_ to ___train___
* Greedy algorithm (local optima)
* Very easy to overfit

## How to build a Decision Tree
How to predict with a decision tree it pretty clear: you just answer the questions and follow the path to the appropriate *leaf* node. But how do we build a decision tree? How do we determine which feature we should split on? This is the crux of the decision tree algorithm.

We will start by dealing with a particular type of decision tree, where we only do binary splits. To do a binary split:

* for a categorical variable, choose either value or not value (e.g. sunny or not sunny)
* for a continuous variable, choose a threshold and do > or <= the value (e.g. temperature <75 or >=75)



### Information Gain
In order to pick which feature to split on, we need a way of measuring how good the split is. This is what *information gain* is for. The *gini impurity* is another alternative, which we'll discuss later.

First, we need to discuss *entropy*. The entropy of a set is a measure of the amount of disorder. Intuitively, if a set has all the same labels, that'll have low entropy and if it has a mix of labels, that's high entropy. We would like to create splits that minimize the entropy in each size. If our splits do a good job splitting along the boundary between classes, they have more predictive power.

The intuition of entropy is more important than the actual function, which follows.

![entropy](images/entropy.png)

Here, P(c) is the percent of the group that belongs to a given class.

If you have a collection of datapoints, the entropy will be large when they are evenly distributed across the classes and small when they are mostly the same class. Here's a graph to demonstrate what entropy looks like:

![entropy](images/entropy_graph_small.png)

So we would like splits that minimize entropy. We use *information gain* to determine the best split:

![information gain](images/gain.png)

Here, S is the original set and D is the splitting of the set (a partition). Each V is a subset of S. All of the V's are disjoint and make up S.


### Gini impurity

The *Gini impurity* is another way of measuring which split is the best. It's a measure of this probability:

* Take a random element from the set
* Label it randomly according to the distribution of labels in the set
* What is the probability that it is labeled incorrectly?

This is the gini impurity:

![gini impurity](images/gini.png)


### Tree Construction
To build our tree, we use a brute force method. We try literally every possibility for splitting at each node and choose the one with the best information gain. Here's the pseudocode for building a Decision Tree:

```
function BuildTree:
    If every item in the dataset is in the same class
    or there is no feature left to split the data:
        return a leaf node with the class label
    Else:
        find the best feature and value to split the data 
        split the dataset
        create a node
        for each split
            call BuildTree and add the result as a child of the node
        return node
```


## Application

Use the `plot_decision_boundary()` function and the starter code provided to compare Logistic Regression, KNN, and DecisionTrees on the `grad.csv` dataset.

1) Load the .csv file from `./data/grad.csv` into a suitable format. You can use pandas if you like or np.loadtxt() (you'll have to remove the labels) or roll your own. Only use the ['gre', 'gpa'] features. This version of `plot_decision_boundary` doesn't work with more features.

2) You will need to use StandardScaler() from the sklearn.preprocessing library:

`scaler = StandardScaler()
 scaler.fit(X)
 X = scaler.transform(X)`

3) Use your best LogisticRegression from earlier and your best KNN.

4) Now try out the DecisionTreeClassifier found in the [tree module](http://scikit-learn.org/stable/modules/classes.html#module-sklearn.tree) Plot these boundaries. What do you observe?

5) Based on the boundaries you find, which of the three is the better classifier? Does this conclusion seem "right" to you?

6) Run a final `train_test_split()` and get a final `cross_val_score()` on the DecisionTreeClassifier. Both are found in the [model_selection library](http://scikit-learn.org/stable/modules/classes.html#module-sklearn.model_selection)

In [ ]:
# Create color maps
cmap_light = ListedColormap(['#FFAAAA', '#AAFFAA', '#AAAAFF'])
cmap_bold = ListedColormap(['#FF0000', '#00FF00', '#0000FF'])

def plot_decision_boundary(clf,X,y,cmap_bold, cmap_light,h = .02):
    x_min, x_max = X[:, 0].min() - 1, X[:, 0].max() + 1
    y_min, y_max = X[:, 1].min() - 1, X[:, 1].max() + 1
    xx, yy = np.meshgrid(np.arange(x_min, x_max, h),
                         np.arange(y_min, y_max, h))
    Z = clf.predict(np.c_[xx.ravel(), yy.ravel()])

    # Put the result into a color plot
    Z = Z.reshape(xx.shape)
    plt.figure()
    plt.pcolormesh(xx, yy, Z, cmap=cmap_light)

    # Plot also the training points
    plt.scatter(X[:, 0], X[:, 1], c=y, cmap=cmap_bold,
                edgecolor='k', s=20)
    plt.xlim(xx.min(), xx.max())
    plt.ylim(yy.min(), yy.max())
    return

## Regression Trees

Like many - if not most - standard classifiers, trees and by extension, forests, can be converted into regressors and thus used for continuous predictions. In the case of single trees, the tree parses the inputs into large domains, and produces a single output (based on the training data) as an output of that domain (where the domain is defined by the splits).

In the below figures ([obtained from lecture notes here](http://www.stat.cmu.edu/~cshalizi/350-2006/lecture-10.pdf)) we see the construction of trees for predicting car price. Below is a standard tree regressed against horsepower and wheelbase: 

![tree1](./images/Wheelbase_tree_2.png)

The following figure is typically called a *partititioning* of the data, where the boundaries of each domain are determined based on the branch points of the tree. When the regression tree is built, the y_train values of each member of the leaf splits are averaged to create a single output. Although this partition is visualizable in two dimensions, additional regressor variables create further dimensions to the partition:

![partition](./images/Tree_Parsing_1.png)


It's worth noting that if multiple predictor variables (features) are equally good at determining the outcome, the split chosen at that branch level is determined by chance. This is the essential concept behind the notion of *feature importance*. Here is the same tree, regressed against weight instead of wheelbase at the second level:


![tree1](./images/Weight_tree_1.png)


#### Regression Forests

Just as we do with random forest, a regression forest simply averages the predictions of its component trees (thus averaging a lot).